# Laboratorio 4 - Parte 1

### Redes Neuronales Artificiales: MLP

### 2019-I

#### Profesor: Julián D. Arias Londoño
#### julian.ariasl@udea.edu.co


## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

#### Primer integrante: Daniel Santa Rendón
#### Documento: 1040050021


#### Segundo integrante: Daniel Torres González
#### Documento: 1020492000

In [1]:
from __future__ import division
%matplotlib inline
import numpy as np
import math
import matplotlib.pyplot as plt


#Algunas advertencias que queremos evitar
import warnings
warnings.filterwarnings("always")

## Indicaciones

Este ejercicio tiene como objetivo implementar una red neuronal artificial de tipo perceptrón multicapa (MLP) para resolver un problema de regresión. Usaremos la librería sklearn. Consulte todo lo relacionado con la definición de hiperparámetros, los métodos para el entrenamiento y la predicción de nuevas muestras en el siguiente enlace: http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor

Para este ejercicio usaremos la base de datos sobre calidad del aire, que ha sido usada en laboratorios previos, pero en este caso trataremos de predecir dos variables en lugar de una, es decir, abordaremos un problema de múltiples salidas.

In [2]:
#cargamos la bd que está en un archivo .data y ahora la podemos manejar de forma matricial
db = np.loadtxt('DB/AirQuality.data',delimiter='\t')  # Assuming tab-delimiter

#Esta es la base de datos AirQuality del UCI Machine Learning Repository. En la siguiente URL se encuentra toda
#la descripción de la base de datos y la contextualización del problema.
#https://archive.ics.uci.edu/ml/datasets/Air+Quality#

X = db[:,0:11]
Y = db[:,11:13]

In [3]:
#Mean Absolute Percentage Error para los problemas de regresión
def MAPE(Y_est,Y):
    N = np.size(Y)
    mape = np.sum(abs((Y_est.reshape(N,1) - Y.reshape(N,1))/Y.reshape(N,1)))/N
    return mape 

## Ejercicio 1

Complete el script siguiente con el código necesario para usar una red neuronal tipo MLP para solucionar el problema de regresión propuesto. Como función de activación en las capas ocultas use la función 'tanh'. Ajuste el número máximo de épocas a 500.

In [4]:
from numpy import random
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import ShuffleSplit
from sklearn import preprocessing
import numpy.matlib as matlib
#Validamos el modelo
Folds = 4
random.seed(19680801)
ErrorY1 = np.zeros(Folds)
ErrorY2 = np.zeros(Folds)
ErrorT = np.zeros(Folds)
ss = ShuffleSplit(n_splits=Folds, test_size=0.3)
mlp=MLPRegressor(activation='tanh',max_iter = 500)
j = 0
for train, test in ss.split(X):
    Xtrain = X[train,:]
    Ytrain = Y[train,:]
    Xtest = X[test,:]
    Ytest = Y[test,:]
    
    #Normalizamos los datos
    media = np.mean(Xtrain,axis=0)
    desvia = np.std(Xtrain,axis=0)
    Xtrain = preprocessing.scale(Xtrain)
    Xtest = (Xtest - np.matlib.repmat(media, Xtest.shape[0], 1))/np.matlib.repmat(desvia, Xtest.shape[0], 1)
    
    #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
    mlp.fit(Xtrain,Ytrain)
    
    
    #Use para el modelo para hacer predicciones sobre el conjunto Xtest
    Yest = mlp.predict(Xtest)

    #Mida el error MAPE para cada una de las dos salidas
    ErrorY1[j] = MAPE(Yest[:,0],Ytest[:,0])
    ErrorY2[j] = MAPE(Yest[:,1],Ytest[:,1])
    ErrorT[j] = (ErrorY1[j] + ErrorY2[j])/2
    j += 1

print('MAPE salida 1 = ' + str(np.mean(ErrorY1)) + '+-' + str(np.std(ErrorY1)))
print('MAPE salida 2 = ' + str(np.mean(ErrorY2)) + '+-' + str(np.std(ErrorY2)))
print('MAPE total = '    + str(np.mean(ErrorT)) + '+-' + str(np.std(ErrorT)))

C:\Users\ALEJO\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\ALEJO\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\ALEJO\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\ALEJO\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\ALEJO\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 19

MAPE salida 1 = 0.06810714099629027+-0.0003222020370612107
MAPE salida 2 = 0.715178215040023+-0.029372206279973955
MAPE total = 0.39164267801815666+-0.014724641547376401


C:\Users\ALEJO\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


## Ejercicio 2

Una vez completado el código anterior. Realice los experimentos necesarios para completar la tabla siguiente:

In [7]:
def prueba(nCO,nPC):
    Folds = 4
    random.seed(19680801)
    ErrorY1 = np.zeros(Folds)
    ErrorY2 = np.zeros(Folds)
    ErrorT = np.zeros(Folds)
    ss = ShuffleSplit(n_splits=Folds, test_size=0.3)
    if(nCO==1):
        MLPRModel=MLPRegressor(activation='tanh',max_iter = 500,hidden_layer_sizes=(nPC))
    else:
        MLPRModel=MLPRegressor(activation='tanh',max_iter = 500,hidden_layer_sizes=(nPC,nPC))
    j = 0
    for train, test in ss.split(X):
        Xtrain = X[train,:]
        Ytrain = Y[train,:]
        Xtest = X[test,:]
        Ytest = Y[test,:]

        #Normalizamos los datos
        media = np.mean(Xtrain,axis=0)
        desvia = np.std(Xtrain,axis=0)
        Xtrain = preprocessing.scale(Xtrain)
        Xtest = (Xtest - np.matlib.repmat(media, Xtest.shape[0], 1))/np.matlib.repmat(desvia, Xtest.shape[0], 1)

        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        MLPRModel.fit(Xtrain,Ytrain)
        

        #Use para el modelo para hacer predicciones sobre el conjunto Xtest
        Yest = MLPRModel.predict(Xtest)

        #Mida el error MAPE para cada una de las dos salidas
        ErrorY1[j] = MAPE(Yest[:,0],Ytest[:,0])
        ErrorY2[j] = MAPE(Yest[:,1],Ytest[:,1])
        j += 1
    m1 = np.mean(ErrorY1) 
    icm1 = np.std(ErrorY1)
    m2 = np.mean(ErrorY2)
    icm2 = np.std(ErrorY2)
    return m1, icm1, m2,icm2

In [9]:
import pandas as pd
import qgrid

df_types = pd.DataFrame({
    'N. de capas ocultas' : pd.Series([1,1,1,1,1,2,2,2,2,2]),
    'Neuronas por capa' : pd.Series([20,24,28,32,36,20,24,28,32,36])})
df_types["MAPE salida 1"] = ""
df_types["IC MAPE salida 1"] = ""
df_types["MAPE salida 2"] = ""
df_types["IC MAPE salida 2"] = ""
df_types.set_index(['N. de capas ocultas','Neuronas por capa'], inplace=True)

for i in df_types.index:
    
    m1, icm1, m2, icm2 = prueba(i[0], i[1])
    
    df_types["MAPE salida 1"][i] = m1
    df_types["IC MAPE salida 1"][i] = icm1
    df_types["MAPE salida 2"][i] = m2
    df_types["IC MAPE salida 2"][i] = icm2

qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)

C:\Users\ALEJO\Anaconda3\lib\site-packages\ipywidgets\widgets\widget.py:281: DeprecationWarning: Widget registration using a string name has been deprecated. Widget registration now uses a plain `@register` decorator.
  warnings.warn("Widget registration using a string name has been deprecated. Widget registration now uses a plain `@register` decorator.", DeprecationWarning)
C:\Users\ALEJO\Anaconda3\lib\site-packages\qgrid\grid.py:573: DeprecationWarning: metadata {'sync': True} was set from the constructor. With traitlets 4.1, metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  _df_json = Unicode('', sync=True)
C:\Users\ALEJO\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\ALEJO\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_percep

C:\Users\ALEJO\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\ALEJO\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\ALEJO\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\Users\ALEJO\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, Converge

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [10]:
qgrid_widget.get_changed_df()

MAPE salida 1 IC MAPE salida 1  \
N. de capas ocultas Neuronas por capa                                  
1                   20                      0.08623       0.00195511   
                    24                     0.081843       0.00186655   
                    28                    0.0772476       0.00135356   
                    32                    0.0768586       0.00173177   
                    36                    0.0750917      0.000896848   
2                   20                    0.0714244       0.00312684   
                    24                    0.0664214       0.00231797   
                    28                    0.0652746      0.000813113   
                    32                    0.0636307       0.00107194   
                    36                    0.0625756       0.00212347   

                                      MAPE salida 2 IC MAPE salida 2  
N. de capas ocultas Neuronas por capa                                 
1                   20                      2.39858         0.158548  
                    24                      2.06149         0.109601  
                    28                      1.73297        0.0694282  
                    32                      1.51695        0.0508611  
                    36                      1.50108        0.0389907  
2                   20                       1.2208        0.0997434  
                    24                     0.984584         0.107353  
                    28                      0.82862        0.0993152  
                    32                     0.868463        0.0884185  
                    36                     0.857715         0.013429

<b>Responda</b>:

2.1 ¿Qué tipo de función de activación usa el modelo en la capa de salida?: 

In [12]:
mlp.out_activation_ 

'identity'

La función de activacipon de la capa de salida es *identity*, es una activación no operativa, útil para implementar un cuello de botella lineal, devuelve f (x) = x

## Ejercicio 3.

A continuación se leen los datos de un problema de clasificación. El problema corresponde a la clasifiación de dígitos escritos a mano. Usaremos únicamente 4 de las 10 clases disponibles. Los datos fueron preprocesados para reducir el número de características. La técnica usada será analizada más adelante en el curso.

In [13]:
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
digits = load_digits(n_class=4)

#--------- preprocesamiento--------------------
pca = PCA(0.99, whiten=True)
data = pca.fit_transform(digits.data)

#---------- Datos a usar ----------------------
X = data
Y = digits.target

C:\Users\ALEJO\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\ALEJO\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\ALEJO\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\ALEJO\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\ALEJO\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 19

Este ejercicio tiene como objetivo implementar una red neuronal artificial de tipo perceptrón multicapa (MLP) para resolver un problema de clasificación. Usaremos la librería sklearn. Consulte todo lo relacionado con la definición de hiperparámetros, los métodos para el entrenamiento y la predicción de nuevas muestras en el siguiente enlace: http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier

Complete el script siguiente con el código necesario para usar una red neuronal tipo MLP para solucionar el problema de clasificación propuesto. Como función de activación en las capas ocultas use la función 'tanh'. Ajuste el número máximo de épocas a 500.

In [14]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold

Folds = 4
random.seed(19680801)
EficienciaTrain = np.zeros(Folds)
EficienciaVal = np.zeros(Folds)
skf = StratifiedKFold(n_splits=Folds)
j = 0
mlp = MLPClassifier(activation='tanh',max_iter = 500)
for train, test in skf.split(X, Y):
    Xtrain = X[train,:]
    Ytrain = Y[train]
    Xtest = X[test,:]
    Ytest = Y[test]
    
    #Normalizamos los datos
    media = np.mean(Xtrain)
    desvia = np.std(Xtrain)
    Xtrain = preprocessing.scale(Xtrain)
    Xtest = (Xtest - np.matlib.repmat(media, Xtest.shape[0], 1))/np.matlib.repmat(desvia, Xtest.shape[0], 1)
    
    #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
    mlp.fit(Xtrain, Ytrain)
    
    #Validación con las muestras de entrenamiento
    Ytrain_pred = mlp.predict(Xtrain)

    #Validación con las muestras de test    
    Yest = mlp.predict(Xtest)
    
    #Evaluamos las predicciones del modelo con los datos de test
    EficienciaTrain[j] = np.mean(Ytrain_pred == Ytrain)
    EficienciaVal[j] = np.mean(Yest == Ytest)
    j += 1
        
print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))

Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.959556601824483+-0.03261581094228954


In [24]:
def prueba2(nCO, nPC):
    if(nCO==1):
        mlp=MLPClassifier(activation='tanh',max_iter = 500,hidden_layer_sizes=(nPC))
    else:
        mlp=MLPClassifier(activation='tanh',max_iter = 500,hidden_layer_sizes=(nPC,nPC))
    Folds = 4
    random.seed(19680801)
    EficienciaTrain = np.zeros(Folds)
    EficienciaVal = np.zeros(Folds)
    skf = StratifiedKFold(n_splits=Folds)
    j = 0
    for train, test in skf.split(X, Y):
        Xtrain = X[train,:]
        Ytrain = Y[train]
        Xtest = X[test,:]
        Ytest = Y[test]

        #Normalizamos los datos
        media = np.mean(Xtrain)
        desvia = np.std(Xtrain)
        Xtrain = preprocessing.scale(Xtrain)
        Xtest = (Xtest - np.matlib.repmat(media, Xtest.shape[0], 1))/np.matlib.repmat(desvia, Xtest.shape[0], 1)

        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        mlp.fit(Xtrain, Ytrain)

        #Validación con las muestras de entrenamiento
        Ytrain_pred = mlp.predict(Xtrain)

        #Validación con las muestras de test    
        Yest = mlp.predict(Xtest)

        #Evaluamos las predicciones del modelo con los datos de test
        EficienciaTrain[j] = np.mean(Ytrain_pred == Ytrain)
        EficienciaVal[j] = np.mean(Yest == Ytest)
        j += 1
        
    m = str(np.mean(EficienciaVal)) 
    icm = str(np.std(EficienciaVal))
    return m, icm

## Ejercicio 4

Una vez completado el código realice los experimentos necesarios para llenar la siguiente tabla:

In [25]:
df_types = pd.DataFrame({
    'N. de capas ocultas' : pd.Series([1,1,1,1,1,2,2,2,2,2]),
    'Neuronas por capa' : pd.Series([20,24,28,32,36,20,24,28,32,36])})
df_types["Eficiencia en validacion"] = ""
df_types["Intervalo de confianza"] = ""
df_types.set_index(['N. de capas ocultas','Neuronas por capa'], inplace=True)
"""
  df_types["Eficiencia en validacion"][2,24] = 0.702417	
  df_types["Intervalo de confianza"][2,24] = 0.027835
"""
#Ponga acá su código, use como valores de referencia los valores anteriores para comprobar su código
for i in df_types.index:
    
    m1, icm1 = prueba2(i[0], i[1])
    
    df_types["Eficiencia en validacion"][i] = m1
    df_types["Intervalo de confianza"][i] = icm1
    
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [26]:
qgrid_widget.get_changed_df()

Eficiencia en validacion  \
N. de capas ocultas Neuronas por capa                            
1                   20                      0.9567941737434934   
                    24                      0.9623501551552112   
                    28                      0.9637852603042869   
                    32                      0.9651745793525728   
                    36                      0.9651201751904246   
2                   20                      0.9679914115957103   
                    24                      0.9568715967184709   
                    28                      0.9651897546863094   
                    32                      0.9610155003686535   
                    36                      0.9680065897876841   

                                      Intervalo de confianza  
N. de capas ocultas Neuronas por capa                         
1                   20                  0.026823128593228423  
                    24                  0.033708835273301894  
                    28                   0.03034887331847802  
                    32                  0.024554487450665758  
                    36                  0.031155322934462944  
2                   20                  0.026517842006687012  
                    24                  0.029363263032099948  
                    28                   0.02842124243349742  
                    32                  0.025302076371385318  
                    36                  0.021431415557815768

<b>Responda</b>:

4.1 ¿Qué tipo de función de activación usa el modelo en la capa de salida?: 


In [27]:
mlp.out_activation_ 

'softmax'

Es softmax porque es un problema de múltiples clases, lo que hace es activar la salida con mayor probabilidad de que la muestra pertenezca a esa clase. 


4.2 ¿Cuántas neuronas en la capa de salida tiene el modelo?¿Porqué debe tener ese número?

In [28]:
mlp.n_outputs_

4

Son 4 salidas porque es un problema de múltiples clases, entonces es una salida para cada clase